### Run the following chunk of code to import any libraries and packages needed to successfully run the rest of the script.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os as os
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2 as cv2

### Run the following chunk of code to extract precision, recall, and F1 score values for various confidence threshold values.

The user will need to update the base_Directory variable to their own directory layout. Users should also keep in mind the general directory structure of this script.

In [ ]:
confidence_Values = np.arange(0.5,1,0.01)

base_Directory = ""
network = tf.keras.models.load_model(base_Directory + "Training/Phase_2/Trained_Model/Phase_Two_Trained_Network.h5")
positive_Directory = base_Directory + "Training/Phase_2/Raw_Training_Cells_Train_Split/Test/Car_T/"
negative_Directory = base_Directory + "Training/Phase_2/Raw_Training_Cells_Train_Split/Test/Other/"
f1_Scores = []
recalls = []
precisions = []
for confidence in confidence_Values:
    print("Analyzing With Confidence Bound " + str(confidence) + ":")
    true_Values = []
    guesses = []
    for file in os.listdir(negative_Directory):
        current_Image = plt.imread(negative_Directory + file)
        if np.max(current_Image) == int(np.max(current_Image)) and len(str(np.max(current_Image))) == len(str(int(np.max(current_Image)))):
            current_Image = current_Image.copy()/255.
        if len(np.shape(current_Image)) == 2:
            current_Image = cv2.cvtColor(current_Image.copy(), cv2.COLOR_GRAY2RGB)
        if np.shape(current_Image)[2] == 4:
            current_Image = current_Image.copy()[:,:,0:3]
        current_Image = preprocess_input(255*current_Image.copy())
        current_Image = np.reshape(current_Image.copy(),(1,40,40,3))
        prediction = network.predict(current_Image, verbose = 0)
        true_Values.append("negative")
        if np.argmax(prediction) == 0 and prediction[0,np.argmax(prediction)] > confidence:
            guesses.append("positive")
        else:
            guesses.append("negative")
    truths = [guesses[i] == true_Values[i] for i in range(len(guesses))]
    true_Negatives = truths.count(True)
    false_Positives = truths.count(False)
    print("True Negative Rate = " + str(true_Negatives/len(truths)))
    print("False Positive Rate = " + str(false_Positives/len(truths)))
    
    true_Values = []
    guesses = []
    
    for file in os.listdir(positive_Directory):
        current_Image = plt.imread(positive_Directory + file)
        if np.max(current_Image) == int(np.max(current_Image)) and len(str(np.max(current_Image))) == len(str(int(np.max(current_Image)))):
            current_Image = current_Image.copy()/255.
        if len(np.shape(current_Image)) == 2:
            current_Image = cv2.cvtColor(current_Image.copy(), cv2.COLOR_GRAY2RGB)
        if np.shape(current_Image)[2] == 4:
            current_Image = current_Image.copy()[:,:,0:3]
        current_Image = preprocess_input(255*current_Image.copy())
        current_Image = np.reshape(current_Image.copy(),(1,40,40,3))
        prediction = network.predict(current_Image, verbose = 0)
        true_Values.append("positive")
        if np.argmax(prediction) == 0 and prediction[0,np.argmax(prediction)] > confidence:
            guesses.append("positive")
        else:
            guesses.append("negative")
    truths = [guesses[i] == true_Values[i] for i in range(len(guesses))]
    true_Positives = truths.count(True)
    false_Negatives = truths.count(False)
    recall = true_Positives/(true_Positives + false_Negatives)
    precision = true_Positives/(true_Positives + false_Positives)
    f1 = true_Positives/(true_Positives + 1/2*(false_Positives + false_Negatives))
    print("True Positive Rate = " + str(true_Positives/len(truths)))
    print("False Negative Rate = " + str(false_Negatives/len(truths)))
    print("Precision = " + str(precision))
    print("Recall = " + str(recall))
    print("F1 = " + str(f1))
    print("")
    f1_Scores.append(f1)
    recalls.append(recall)
    precisions.append(precision)

### Run the following chunk of code to save the generated precisions, recalls, and F1 scores to .npy numpy arrays for later analysis.

In [ ]:
np.save(base_Directory + "Analysis/Precisions/Precisions_Test.npy",precisions)
np.save(base_Directory + "Analysis/Recalls/Recalls_Test.npy",recalls)
np.save(base_Directory + "Analysis/F1_Scores/F1_Scores_Test.npy",f1_Scores)